In [24]:
import os
import pandas as pd
import requests

We will read the input excel file that we got  from client. this file has the start and end geocoordinates of the different plants. We have to assess different route between these plants

In [25]:
input_from_client = pd.read_excel(r'C:\Users\Mohit\Desktop\Code\Projects\TruckVsDrivingRouteAnalysis\Input_from_client.xlsx')

In [26]:
input_from_client.head()
# input_from_client.info()

,uid,start,end,start_lat,start_lon,end_lat,end_lon
0,1,Bidadi,Ambikapur,12.782286,77.424975,23.112297,83.182082
1,2,Bidadi,Bhilai,12.782286,77.424975,21.208241,81.379906
2,3,Bidadi,Raipur,12.782286,77.424975,21.219474,81.631294
3,4,Becharaji Plant,JHANSI,23.495345,72.022300,25.445902,78.589940
4,5,Becharaji Plant,SHAHDOL - JBP,23.495345,72.022300,23.302021,81.369715


Now we will make different Route API call with Driving and Trucking profile to chech how different these 2 profiles are 

In [32]:
# # Define the base URL for the API
# routing_API_url = 'https://apis.mapmyindia.com/advancedmaps/v1/3q7gfp72nrkhdjvi19p43hp78f5k36vh/route_eta/driving/'
                  

# # Define the coordinates for the route
# coordinates = '77.424975,12.782286;83.182082,23.112297'

# # Define the parameters for the API request
# params = {
#     'geometries': 'polyline6',
#     'overview': 'full'
# }

# # Send the API request and store the response
# routing_API_response = requests.get(routing_API_url + coordinates, params=params)

# routing_API_data = routing_API_response.json()

# # print(routing_API_data['routes']['geometry']
# print(routing_API_data['routes'][0]['geometry'])

# print(routing_API_data['routes'][0]['duration'])
# print(routing_API_data['routes'][0]['distance'])


In [33]:
# # URL of the Route API call in driving profile
# routing_API_Driving_url = 'https://apis.mapmyindia.com/advancedmaps/v1/3q7gfp72nrkhdjvi19p43hp78f5k36vh/route_eta/driving/'

# # Define the parameters for the API request
# params = {
#     'geometries': 'polyline6',
#     'overview': 'full'
# }

# for index in input_from_client.index:
    
#     coordinates = str(input_from_client.at[index,'start_lon']) + ',' + str(input_from_client.at[index,'start_lat']) + ';' + str(input_from_client.at[index,'end_lon']) + ',' + str(input_from_client.at[index,'end_lat'])+ '?'

#     routing_API_response = requests.get(routing_API_Driving_url + coordinates, params=params)

#     try:

#         routing_API_data = routing_API_response.json()

#         input_from_client.loc[index, 'Duration(s)'] = routing_API_data['routes'][0]['duration']
#         input_from_client.loc[index, 'Distance(m)'] = routing_API_data['routes'][0]['distance']
#         input_from_client.loc[index, 'Driving Geometry'] = routing_API_data['routes'][0]['geometry']

#         print('The API call for request {index} was  successful'.format(index=index))
    
#     except:
#         print('There was an error with the API call for request {index}'} .format(index=index))


In [ ]:
# URL of the Route API call in driving profile
routing_API_Driving_url = 'https://apis.mapmyindia.com/advancedmaps/v1/3q7gfp72nrkhdjvi19p43hp78f5k36vh/route_adv/driving/'

# Define the parameters for the API request
params = {
    'geometries': 'polyline6',
    'overview': 'full'
}

for index, row in input_from_client.iterrows():
    
    coordinates = f"{row['start_lon']},{row['start_lat']};{row['end_lon']},{row['end_lat']}?"
    print(coordinates)

    routing_API_response = requests.get(routing_API_Driving_url + coordinates, params=params)

    try:
        routing_API_data = routing_API_response.json()
        input_from_client.loc[index, 'Driving_Duration(s)'] = routing_API_data['routes'][0]['duration']
        input_from_client.loc[index, 'Driving_Distance(m)'] = routing_API_data['routes'][0]['distance']
        input_from_client.loc[index, 'Driving_Geometry'] = routing_API_data['routes'][0]['geometry']

        # filename = f"{index}.txt"
        # with open(filename, 'w', encoding='utf-8') as text_file:
        #     text_file.write(routing_API_data['routes'][0]['geometry'])

        # Assuming routing_API_data['routes'][0]['geometry'] contains "\\" characters
        geometry_raw = r"{}".format(routing_API_data['routes'][0]['geometry'])

        # Assigning the raw string representation to the DataFrame
        input_from_client.loc[index, 'New_Driving_Geometry'] = geometry_raw


        print(f"The API call for request {index} was successful")
    except:
        print(f"There was an error with the API call for request {index}")


input_from_client.head()

In [28]:
# print(input_from_client.loc[0,'Driving_Geometry'])
input_from_client.head()

print(len(input_from_client.loc[0,'Driving_Geometry']))
print(len(input_from_client.loc[0,'New_Driving_Geometry']))




128316
128316


In [7]:
filename = f"Escaped.txt"
with open(filename, 'w', encoding='utf-8') as text_file:
    text_file.write(input_from_client.loc[0,'Driving_Geometry'])

In [29]:
filename2 = f"unescapped.txt"
with open(filename2, 'w', encoding='utf-8') as text_file:
    text_file.write(input_from_client.loc[0,'New_Driving_Geometry'])

In [13]:
new_string = r"@g\\nSg\\nSce@tXgI|Ewe@dYyg@~Zyg@~ZkQvKoOnIkb@zVwSxMsZnQsh@`\\sh@b\\wBvAom@d"


In [14]:
print(new_string)

@g\\nSg\\nSce@tXgI|Ewe@dYyg@~Zyg@~ZkQvKoOnIkb@zVwSxMsZnQsh@`\\sh@b\\wBvAom@d
